In [ ]:
# Rice Type Detection Training Notebook

#This notebook demonstrates how to train a deep learning model to classify rice grain types using the Rice Image Dataset from Kaggle. We will use transfer learning with MobileNetV2, data augmentation, and evaluate the model's performance.

'''---

**Steps covered in this notebook:**
1. Import required libraries
2. Load and explore the dataset
3. Data preprocessing and augmentation
4. Build and train the model (using MobileNetV2)
5. Evaluate and save the model'''
pip install numpy
pip install pandas
pip install tensorflow==2.3.2
pip install keras==2.3.1
pip install Flask



: 

In [ ]:
# Import Required Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
import pathlib
import cv2  # Added for image reading


: 

In [ ]:
# Set up the data directory and list sample files for each rice type

data_dir = "../input/rice-image-dataset/Rice_Image_Dataset"  # Update path as needed

data_dir = pathlib.Path(data_dir)

# List a few sample images from each class (limit to 600 for demonstration)
arborio = list(data_dir.glob('Arborio/*'))[:600]
basmati = list(data_dir.glob('Basmati/*'))[:600]
ipsala = list(data_dir.glob('Ipsala/*'))[:600]
jasmine = list(data_dir.glob('Jasmine/*'))[:600]
karacadag = list(data_dir.glob('Karacadag/*'))[:600]

# Display counts for each class
print('Arborio:', len(arborio))
print('Basmati:', len(basmati))
print('Ipsala:', len(ipsala))
print('Jasmine:', len(jasmine))
print('Karacadag:', len(karacadag))

In [ ]:
# Contains the images path
df_images = {
    'arborio': arborio,
    'basmati': basmati,
    'ipsala': ipsala,
    'jasmine': jasmine,
    'karacadag': karacadag
}

# Contains numerical labels for the categories
df_labels = {
    'arborio': 0,
    'basmati': 1,
    'ipsala': 2,
    'jasmine': 3,
    'karacadag': 4
}


In [ ]:
# Load images and labels into X and y lists
X = []
y = []
img_size = 250  # Assuming images are 250x250 as shown in the screenshot

for label_name, images in df_images.items():
    for img_path in images:
        img = cv2.imread(str(img_path))
        img = cv2.resize(img, (img_size, img_size))
        X.append(img)
        y.append(df_labels[label_name])

print(f"Loaded {len(X)} images.")

In [ ]:
# Standardizing
X = np.array(X)
X = X / 255
y = np.array(y)

# Separating data into training, test and validation sets
from sklearn.model_selection import train_test_split
X_train, X_test_val, y_train, y_test_val = train_test_split(X, y)
X_test, X_val, y_test, y_val = train_test_split(X_test_val, y_test_val)


In [ ]:
# Visualize one sample image from each rice category
fig, ax = plt.subplots(ncols=5, figsize=(20, 5))
fig.suptitle('Rice Category')

arborio_image = cv2.imread(str(arborio[0]))
basmati_image = cv2.imread(str(basmati[0]))
ipsala_image = cv2.imread(str(ipsala[0]))
jasmine_image = cv2.imread(str(jasmine[0]))
karacadag_image = cv2.imread(str(karacadag[0]))

ax[0].set_title('arborio')
ax[1].set_title('basmati')
ax[2].set_title('ipsala')
ax[3].set_title('jasmine')
ax[4].set_title('karacadag')

ax[0].imshow(cv2.cvtColor(arborio_image, cv2.COLOR_BGR2RGB))
ax[1].imshow(cv2.cvtColor(basmati_image, cv2.COLOR_BGR2RGB))
ax[2].imshow(cv2.cvtColor(ipsala_image, cv2.COLOR_BGR2RGB))
ax[3].imshow(cv2.cvtColor(jasmine_image, cv2.COLOR_BGR2RGB))
ax[4].imshow(cv2.cvtColor(karacadag_image, cv2.COLOR_BGR2RGB))

plt.show()


In [ ]:
# Build MobileNetV2 as a feature extractor (freeze convolutional layers) using keras.Sequential
input_shape = (224, 224, 3)
num_label = 5  # number of labels

# Resize images if needed
X_train_resized = np.array([cv2.resize(img, (224, 224)) for img in X_train])
X_val_resized = np.array([cv2.resize(img, (224, 224)) for img in X_val])
X_test_resized = np.array([cv2.resize(img, (224, 224)) for img in X_test])

# Load MobileNetV2 with pretrained ImageNet weights, exclude top layers
mobile_net = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape, pooling='avg')
mobile_net.trainable = False  # Freeze all convolutional layers

model = keras.Sequential([
    mobile_net,
    keras.layers.Dense(num_label)
])

model.compile(
    optimizer="adam",
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['acc']
)
model.summary()


In [ ]:
# Train the model
history = model.fit(
    X_train_resized, y_train,
    epochs=10,
    validation_data=(X_val_resized, y_val)
)


In [ ]:
# Evaluate the model using classification_report
from sklearn.metrics import classification_report

y_pred = model.predict(X_test_resized, batch_size=64, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)

print(classification_report(y_test, y_pred_bool))


In [ ]:
from plotly.offline import iplot, init_notebook_mode
import plotly.express as px
import pandas as pd

init_notebook_mode(connected=True)

acc = pd.DataFrame({'train': history.history['acc'], 'val': history.history['val_acc']})

fig = px.line(acc, x=acc.index, y=acc.columns[0:],'title="Training and Evaluation Accuracy every Epoch" , markers=True)
fig.show()


In [ ]:
# Test the model on a single image

a1 = cv2.imread("../input/rice-image-dataset/Rice_Image_Dataset/Basmati/basmati (10).jpg")
a1 = cv2.resize(a1, (224,224))
a1 = np.array(a1)
a1 = a1/255
a1 = np.expand_dims(a1, 0)
pred = model.predict(a1)
pred = pred.argmax()
pred


In [ ]:
# Save the trained model
model.save("rice.h5")
